# Animating the training of one neural network imitating another

We start with neural network A with random weights.

We produce neural network B with random weights.

We uniformly randomly sample inputs, compute the output from network A, and use the input-output pair as training for network B.

We extract the weights into a one dimensional array then display it as a two dimensional array for visualization purposes.

Does network B learn the weights of network A?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import tensorflow as tf
import math

N=10

def weights_into_square_array(model):
  num_params = model.count_params()
  dim = math.ceil(math.sqrt(num_params))
  pad = np.zeros(shape=(dim*dim-num_params,))
  
  weights = np.concatenate([x.numpy().flatten() for x in model.weights]+[pad],axis=0)  

  # weights = np.sort(weights)

  weights = np.reshape(weights,(dim,dim))

  return weights




#1. Create target nn
target = tf.keras.Sequential([    
    tf.keras.layers.Dense(8, input_shape=(N,), activation='tanh'),
    tf.keras.layers.Dense(8, activation='tanh'),
    # tf.keras.layers.Dense(64, activation='tanh'),
    # tf.keras.layers.Dense(128, activation='tanh'),
    tf.keras.layers.Dense(1,activation='tanh')
])


#2. Create learner nn
learner = tf.keras.Sequential([    
    tf.keras.layers.Dense(8, input_shape=(N,), activation='tanh'),
    tf.keras.layers.Dense(8, activation='tanh'),
    # tf.keras.layers.Dense(64, activation='tanh'),
    # tf.keras.layers.Dense(128, activation='tanh'),
    tf.keras.layers.Dense(1,activation='tanh')
  ])  


fig, (ax1, ax2) = plt.subplots(1,2)


# # plt.subplot(1,2,1)
# ax1.imshow(weights_into_square_array(target))
# # plt.subplot(1,2,2)
# ax2.imshow(weights_into_square_array(learner))

# plt.show()

opt = tf.optimizers.Adam(0.05)

ims = []
for i in range(50):
  #Step learner
  inpt = tf.random.uniform(shape=(50,N,),minval=-1.,maxval=1.)
  outp = target(inpt).numpy()
  with tf.GradientTape() as tape:
    out = learner(inpt)
    error = tf.reduce_sum(tf.square(outp-out))/50.    
    gradients = tape.gradient(error,learner.trainable_weights)  
  opt.apply_gradients(zip(gradients, learner.trainable_variables))

  #Draw weights
  im = ax1.imshow(weights_into_square_array(target),animated=True)
  im = ax2.imshow(weights_into_square_array(learner),animated=True)
  plt.tight_layout()
  ims.append([im])
  plt.close(fig)

print("Final error ",error.numpy())
ani = animation.ArtistAnimation(fig, ims, interval=100, repeat_delay=200)

HTML(ani.to_html5_video())
    

Final error  0.0059339358


<Figure size 432x288 with 0 Axes>

### Conclusion

From various experiments, it would appear the weights do not converge between the two networks although they appear to converge functionally. This is not too surprising since multiple weights can implement the same function.